In [2]:
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from math import log, factorial
from scipy.optimize import minimize

In [12]:
vax = pd.read_csv("/Users/giovannimaya/Downloads/data/PCC_study_train.csv")
vax = vax.drop(columns='Unnamed: 0')
vax

,vax_status,WBscore,PCCsymp
0,Vaccinated,Improved,Less
1,Vaccinated,Unchanged,Less
2,Vaccinated,Improved,Less
3,Vaccinated,Worsened,Less
4,Vaccinated,Worsened,More
...,...,...,...
70,Unvaccinated,Unchanged,Less
71,Unvaccinated,Worsened,Same
72,Unvaccinated,Improved,Same
73,Unvaccinated,Unchanged,More


In [13]:
vax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   vax_status  75 non-null     object
 1   WBscore     75 non-null     object
 2   PCCsymp     75 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

predictor = vax.columns[:1]
outcome = vax.columns[]


X_train, X_test, y_train, y_test = \
    train_test_split(vax[predictor],
                     vax[outcome], test_size=0.25)
model = GaussianNB()

##########################################
# Train the model / fit model parameters #
##########################################
model.fit(X_train, y_train)

##########################
# Predict with the model #
##########################
y_hat = model.predict(X_test)

##################
# Test the model #
##################
print('First testing datapoint: \n', pd.DataFrame(X_test.iloc[0, :]).T)
print('Model prediction: ', y_hat[0], '\n')




ValueError: could not convert string to float: 'Vaccinated'